**개요 : 영윤의 가장 최근 데이터를 이용해서 같은 방법으로 추천 음식을 출력하고, 비교해봄**

In [213]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [214]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MachineLearningPractice/Colab-Notebooks/FarmKit/foodfile/220721_정영윤_ver3 (영문, 병합 번역).csv')

In [215]:
df.head()

,id,카테고리명,분류기준,식품명,설명,ingre,recipe
0,1,양식,그라탕류,라따뚜이그라탕,"It is a gratin using ratatouille, a traditiona...","Tomato, eggplant, zucchini, penne, tomato sauc...","Prepare tomatoes, eggplant, and zucchini by cu..."
1,2,간식,토스트류,햄치즈토스트,Sliced ​​ham and cheese are placed on bread an...,"White bread, sliced ​​ham, cheddar cheese, can...",Prepare the cabbage and carrots by slicing the...
2,3,밑반찬,건어물류,마른미역자반,It is a side dish of dried seaweed fried in oi...,"Dried seaweed, perilla oil, cooking oil, sugar...",Heat perilla oil and cooking oil in a pan. Whe...
3,4,한식,찌개류,돼지고기된장찌개,It is a stew made by slicing pork and putting ...,"Pork leg meat, green onion, onion, zucchini, r...",Cut the onion and pumpkin into appropriate siz...
4,5,간식,빵류,고구마치즈빵,A sweet-tasting bread topped with boiled sweet...,"Steamed sweet potato, mayonnaise, honey, mozza...","Mix flour, milk, melted butter and salt in a b..."


In [216]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')     # 크게 의미없는 단어들을 제외해준다.
tfidf_matrix = tfidf.fit_transform(df['recipe'])
tfidf_matrix.shape

(2381, 3501)

In [217]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf2 = TfidfVectorizer(stop_words='english')     # 크게 의미없는 단어들을 제외해준다.
tfidf_matrix2 = tfidf2.fit_transform(df['설명'])
tfidf_matrix2.shape

(2381, 2024)

In [218]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)     # 코사인 유사도 계산
cosine_sim

array([[1.        , 0.06803562, 0.0161275 , ..., 0.04574963, 0.02937351,
        0.11821797],
       [0.06803562, 1.        , 0.06208329, ..., 0.03098517, 0.04582339,
        0.15808588],
       [0.0161275 , 0.06208329, 1.        , ..., 0.00780871, 0.0615002 ,
        0.11831635],
       ...,
       [0.04574963, 0.03098517, 0.00780871, ..., 1.        , 0.1143361 ,
        0.02427771],
       [0.02937351, 0.04582339, 0.0615002 , ..., 0.1143361 , 1.        ,
        0.04059509],
       [0.11821797, 0.15808588, 0.11831635, ..., 0.02427771, 0.04059509,
        1.        ]])

In [219]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim2 = linear_kernel(tfidf_matrix2, tfidf_matrix2)     # 코사인 유사도 계산
cosine_sim2

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.07457293,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.18032163,
        0.        ],
       [0.        , 0.07457293, 0.        , ..., 0.18032163, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [220]:
indices = pd.Series(df.index, index=df['식품명']).drop_duplicates()    # 중복 제거
indices

식품명
라따뚜이그라탕        0
햄치즈토스트         1
마른미역자반         2
돼지고기된장찌개       3
고구마치즈빵         4
            ... 
홍루이젠        2376
불고기우동샐러드    2377
바나나경단       2378
바나나로띠       2379
닭가슴살채소덮밥    2380
Length: 2381, dtype: int64

In [221]:
# 음식명을 입력받으면 코사인 유사도를 통해서 가장 유사도가 높은 상위 10개의 음식 목록 반환
def get_recommendations(title, cosine_sim=cosine_sim):

  # 음식명을 통해서 전체 데이터 기준 그 음식의 index 값을 얻기
  idx = indices[title]

  # 코사인 유사도 매트릭스 (cosine_sim) 에서 idx 에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
  sim_scores = list(enumerate(cosine_sim[idx]))

  # 코사인 유사도 기준으로 내림차순 정렬
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # 자기 자신을 제외한 10개의 추천 음식을 슬라이싱
  sim_scores = sim_scores[1:11]

  # 추천 음식목록 10개의 인덱스 정보 추출
  movie_indices = [i[0] for i in sim_scores]

  # 인덱스 정보를 통해 음식명 추출
  return df['식품명'].iloc[movie_indices]

In [222]:
get_recommendations('햄치즈토스트')

1782        양배추샌드위치
1531        달걀채소토스트
2003         모닝샌드위치
1777     참치마요네즈샌드위치
2082       베이컨치즈토스트
109     몬테크리스토 샌드위치
1479         프렌치토스트
62          식빵 누네띠네
44            밤잼토스트
143         치즈갈릭브레드
Name: 식품명, dtype: object

In [223]:
get_recommendations('햄치즈토스트', cosine_sim2)

2376           홍루이젠
109     몬테크리스토 샌드위치
1336         몬테크리스토
1548          허니브레드
1032        레어치즈케이크
997           식빵이자냐
1767        새우그라탕덮밥
1505          크림통식빵
288           닭날개구이
1239            허니윙
Name: 식품명, dtype: object

In [224]:
get_recommendations('고구마치즈빵')

46      제티체크쿠키&제티설기
431            감자피자
1114            공갈빵
173             소금빵
1534       고구마베이컨말이
2063          참깨크래커
2365           시나몬롤
1469         감자찹쌀구이
632             모닝빵
1283           사과파이
Name: 식품명, dtype: object

In [225]:
get_recommendations('고구마치즈빵', cosine_sim2)

1823          더블치즈토스트
1899            치즈마늘빵
2082         베이컨치즈토스트
104              프로틴빵
760               구름빵
837     리얼고구마빵&리얼 감자빵
2118            하이토스트
1505            크림통식빵
2055           고구마치즈볼
109       몬테크리스토 샌드위치
Name: 식품명, dtype: object

In [226]:
get_recommendations('닭가슴살채소덮밥')

1423       닭가슴살채소볶음
98            오트밀카레
643       후루룩닭고기쌀국수
102           순두부인헬
1605         닭가슴살쌈밥
234           오트밀닭죽
1800      닭가슴살오픈토스트
1883         감자칩카나페
1633    통바게트양배추샌드위치
547             NaN
Name: 식품명, dtype: object

In [227]:
get_recommendations('닭가슴살채소덮밥', cosine_sim2)

398          두부김밥
1238        된장리조또
569         김치볶음밥
537           두부밥
206      곤약새우버섯솥밥
2131         오야꼬동
501           곤약밥
643     후루룩닭고기쌀국수
156          꼬막덮밥
1928     데리야끼치킨덮밥
Name: 식품명, dtype: object

**재료를 기준으로 음식을 추천해줌**

In [228]:
df['ingre']

0       Tomato, eggplant, zucchini, penne, tomato sauc...
1       White bread, sliced ​​ham, cheddar cheese, can...
2       Dried seaweed, perilla oil, cooking oil, sugar...
3       Pork leg meat, green onion, onion, zucchini, r...
4       Steamed sweet potato, mayonnaise, honey, mozza...
                              ...                        
2376    Bread, egg, sliced ​​ham, sliced ​​cheese, may...
2377    Beef for bulgogi, udon noodles, lettuce, soy s...
2378    Banana, glutinous rice flour (dry), salt, cast...
2379    Banana, tortilla, egg, butter, condensed milk,...
2380    Boiled chicken breast, cabbage, onion, carrot,...
Name: ingre, Length: 2381, dtype: object

In [229]:
type(df['ingre'][0])

str

In [230]:
df['ingre'] = df['ingre'].apply(lambda x : x.split(','))
df['ingre'] = df['ingre'].apply(lambda x : x[:4])
df['ingre'] = df['ingre'].apply(lambda x : ' '.join(x))
df['ingre']

0                       Tomato  eggplant  zucchini  penne
1       White bread  sliced ​​ham  cheddar cheese  can...
2          Dried seaweed  perilla oil  cooking oil  sugar
3             Pork leg meat  green onion  onion  zucchini
4       Steamed sweet potato  mayonnaise  honey  mozza...
                              ...                        
2376            Bread  egg  sliced ​​ham  sliced ​​cheese
2377    Beef for bulgogi  udon noodles  lettuce  soy s...
2378    Banana  glutinous rice flour (dry)  salt  cast...
2379                        Banana  tortilla  egg  butter
2380        Boiled chicken breast  cabbage  onion  carrot
Name: ingre, Length: 2381, dtype: object

In [231]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix3 = count.fit_transform(df['ingre'])
count_matrix3

<2381x1009 sparse matrix of type '<class 'numpy.int64'>'
	with 14532 stored elements in Compressed Sparse Row format>

In [232]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim3 = cosine_similarity(count_matrix3, count_matrix3)
cosine_sim3

array([[1.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 1.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 1.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 1.       , 0.1767767,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.1767767, 1.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ]])

In [233]:
get_recommendations('데리야끼치킨덮밥', cosine_sim3)

2299     코코넛치킨카레
1045       간장꼬꼬밥
189     치즈닭갈비볶음밥
1171        불닭볶음
1185        닭장떡국
2256        떡닭떡닭
1631       인절미치킨
1684       닭맥적구이
45         오렌지치킨
130          오징어
Name: 식품명, dtype: object

In [234]:
get_recommendations('햄치즈토스트', cosine_sim3)

2082       베이컨치즈토스트
1336         몬테크리스토
274        오믈렛컵&피자컵
480           구름토스트
1906          크로크마담
450          양파햄치즈빵
2376           홍루이젠
1871         치즈폭탄피자
109     몬테크리스토 샌드위치
220           피자토스트
Name: 식품명, dtype: object

In [235]:
get_recommendations('고구마치즈빵', cosine_sim3)

1620      단호박치즈구이
2055       고구마치즈볼
812         고구마경단
844      아코디언꿀고구마
1428      오지치즈후라이
703        고구마핫도그
1910       고구마말랭이
1534     고구마베이컨말이
1560    훈제오리단호박구이
1665     인절미고구마맛탕
Name: 식품명, dtype: object